In [4]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

# Nota: para este caso asumiremos que los datos de entrenamiento son sobre los que trabajaremos

In [32]:
Y= np.array([[3],[1],[8],[3],[5]])
Y

array([[3],
       [1],
       [8],
       [3],
       [5]])

In [26]:
X=np.array([[1,3,5],[1,1,4],[1,5,6],[1,2,4],[1,4,6]])
X

array([[1, 3, 5],
       [1, 1, 4],
       [1, 5, 6],
       [1, 2, 4],
       [1, 4, 6]])

In [16]:
XT_X= np.matmul(np.matrix.transpose(X),X)
XT_X

array([[  5,  15,  25],
       [ 15,  55,  81],
       [ 25,  81, 129]])

In [18]:
# Estos resultados son las cii (formulas libreta) en este caso se va a usar el 26.7 el 1 y el 2.5, ya que sólo se usan los elementos de la diagonal principal
XT_X_inv= np.linalg.inv(XT_X)
XT_X_inv

array([[26.7,  4.5, -8. ],
       [ 4.5,  1. , -1.5],
       [-8. , -1.5,  2.5]])

In [34]:
XT_Y= np.matmul(np.matrix.transpose(X),Y)
XT_Y

array([[ 20],
       [ 76],
       [109]])

In [36]:
betas= np.matmul( XT_X_inv,XT_Y)
betas

array([[ 4. ],
       [ 2.5],
       [-1.5]])

In [38]:
# Cálculo pronosticos de "Y" de acuerdo a los coeficientes de regresión

Y_pred= np.matmul(X,betas)
Y_pred

array([[4. ],
       [0.5],
       [7.5],
       [3. ],
       [5. ]])

In [40]:
# calculo residuales

Resid= Y-Y_pred
Resid

array([[-1.00000000e+00],
       [ 5.00000000e-01],
       [ 5.00000000e-01],
       [-1.33226763e-13],
       [-1.64313008e-13]])

In [42]:
# Opciones de formato numérico para que no se vuelva anotación científica y sí aparezcan decimales

np.set_printoptions(formatter={"float_kind":"{:f}".format})

In [44]:
# calculo residuales

Resid= Y-Y_pred
Resid

array([[-1.000000],
       [0.500000],
       [0.500000],
       [-0.000000],
       [-0.000000]])

In [52]:
# Cálculo de la suma de residuales al cuadrado (RSS)

RSS= float(np.matmul(np.matrix.transpose(Resid),Resid))
RSS

1.4999999999999991

In [56]:
# Cálculo de la suma total de cuadrados los residuales (1.49999 corresponde a un pedazo muy pequeño de lo que no se puede explicar)
# de 28 unidades no se pueden explicar 1.49999 y el resto sí
# FLOAT para que sólo quede el número sin ser un arreglo
TSS= float(np.matmul( np.matrix.transpose(Y),Y)-(len(Y)*( Y.mean()**2)))
TSS

28.0

In [72]:
# Cálculo de coeficiente de determinación r2 (R cuadrada)

R2= float(1-(RSS/TSS))
R2

0.9464285714285715

In [70]:
# Cálculo de coeficiente de determinación r2 (R cuadrada ajustada)
# No se suma +1 la K (número de columnas, ya que ya está la columna de 1 (intercepto)
# baja, pero es más objetiva
R2_ajus= float(1-((RSS/(X.shape[0]-X.shape[1]))/(TSS/(X.shape[0]-1))))
R2_ajuss

0.8928571428571429

In [76]:
# Cálculo de la varianza del error de regresión

s_cuad= RSS/ (len(Y)-X.shape[1])
s_cuad

0.7499999999999996

In [78]:
# Desviación estandar del erros de regresión

import math
s= math.sqrt(s_cuad)
s

0.8660254037844384

In [82]:
# Cálculo de las t estadísticas para cada coeficiente de regresión (betas)
# aquí es donde se vuelven a usar los valores de la diagonal principal cii que se saco en XT_X_inv
result_t=[]
for i in range (0,X.shape[1]):
    t= float(betas[i]/(s*math.sqrt(XT_X_inv[i][i])))
    result_t.append(t)
result_t

[0.893868697538675, 2.8867513459481344, -1.0954451150103264]

# Criterio número 1

In [89]:
# obtener valor crítico de la t de student
import scipy.stats

grados_libertad= len(Y)-X.shape[1]
# la t critica se obtendra a un nivel de confianza del 95% (Alfa=5%) q=.025, ya que es la mitad del 5% en decimal
t_critico= abs(scipy.stats.t.ppf(q=.025, df= grados_libertad))
t_critico
# El area arriba del valor 4.3026 debería de ser de .025 ( se puede observa si se grafica)

4.3026527299112765

In [95]:
# abs se usa para que sea valor absoluto y no afecten números negativos
for i in range(0, X.shape[1]):
    if (abs(result_t[i])>t_critico):
        print("Beta",i, "es significativa") #Aquí se rechaza hipótesis nula HO
    else:
        print("Beta",i,"no es significativa") #Aquí se  no rechaza hipótesis nula HO

Beta 0 no es significativa
Beta 1 no es significativa
Beta 2 no es significativa


# Criterio número 2

In [100]:
# Cálculo valores p
# se multiplica por 2 al final ya que se están buscando el area de adelante y de atrás y al ser simétrica sí funciona
for i in range(0,X.shape[1]):
    print ("Valor p de Beta", i, ":", scipy.stats.t.sf(abs(result_t[i]),df= grados_libertad)*2)

Valor p de Beta 0 : 0.4657159826085253
Valor p de Beta 1 : 0.10197348986612516
Valor p de Beta 2 : 0.38762756430420753


##### si manejamos un nivel Alpha del 5% en ninguno de los casos el valor p es menor, por lo tanto, no se puede rechazar la hipotesis nula y coincide con los primeros resultados

# criterio número 3

In [111]:
# cálcuo de intervalos de confianza del 95% para el verdadero valor del coeficiente de cada beta (variable)
for i in range(0, X.shape[1]):
    print("el valor de Beta", i, "Se encuentra entre", float(betas[i])-t_critico*s*math.sqrt(XT_X_inv[i][i]),
          "y" ,float(betas[i])+t_critico*s*math.sqrt(XT_X_inv[i][i]))

el valor de Beta 0 Se encuentra entre -15.25407049943233 y 23.25407049943251
el valor de Beta 1 Se encuentra entre -1.226206567765626 y 6.226206567765638
el valor de Beta 2 Se encuentra entre -7.391649893208962 y 4.391649893208978


##### al si estar el 0 entre los intervalos de cada beta, no se puede garantizar que la Beta sea distinta a cero
##### por lo tanto, no se puede rechazar la hipótesis nula, pero tampoco se puede aceptar

# Conclusión: ninguna de las varibles independientes es significativamente distinta a cero

# comparación de resultados con reporte automático

In [120]:
import statsmodels.api as sm
regressor= sm.OLS(Y,X).fit()
print(regressor.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.946
Model:                            OLS   Adj. R-squared:                  0.893
Method:                 Least Squares   F-statistic:                     17.67
Date:                Thu, 29 May 2025   Prob (F-statistic):             0.0536
Time:                        19:44:03   Log-Likelihood:                -4.0848
No. Observations:                   5   AIC:                             14.17
Df Residuals:                       2   BIC:                             13.00
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          4.0000      4.475      0.894      0.4

C:\Users\odela\AppData\Local\Programs\Orange\Lib\site-packages\statsmodels\stats\stattools.py:74: ValueWarning: omni_normtest is not valid with less than 8 observations; 5 samples were given.
  warn("omni_normtest is not valid with less than 8 observations; %i "


## al tener R2 tan altos y que los modelos arrojen que las variables no son estadísticamente significativas, se puede deber al que hay una gran correlación entre ambas variables (multicolianealidad) y no pueden coexistir variables correlacionadas

## Sólo se puede quitar una variable a la vez y se repite el análisis

In [122]:
# prueba de multicolinealidad ente varibles
# tienen correlaciones muy altas
data= pd.DataFrame(X)
data2= data.iloc[:,1:3]
data2.corr()

,1,2
1,1.000000,0.948683
2,0.948683,1.000000


#### para saber cual variable quitar, se revisa el valor de p y se quita al que tenga el valor más alto, en este caso la segunda variable con .388

In [140]:
# reporte con sólo la variable que se va a dejar
# np.delete para borrar columnas o renglones de un arreglo, X es el arreglo, luego se pone el número de renglón o columna deseada
# y por último se indica con 1 si es columna y con 0 sí es renglón
X_nueva= np.delete(X,2,1)
X_nueva

array([[1, 3],
       [1, 1],
       [1, 5],
       [1, 2],
       [1, 4]])

In [142]:
regressor= sm.OLS(Y,X_nueva).fit()
print(regressor.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.914
Model:                            OLS   Adj. R-squared:                  0.886
Method:                 Least Squares   F-statistic:                     32.00
Date:                Thu, 29 May 2025   Prob (F-statistic):             0.0109
Time:                        20:00:18   Log-Likelihood:                -5.2598
No. Observations:                   5   AIC:                             14.52
Df Residuals:                       3   BIC:                             13.74
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.8000      0.938     -0.853      0.4

C:\Users\odela\AppData\Local\Programs\Orange\Lib\site-packages\statsmodels\stats\stattools.py:74: ValueWarning: omni_normtest is not valid with less than 8 observations; 5 samples were given.
  warn("omni_normtest is not valid with less than 8 observations; %i "


#### la variable ahora sí es significativa con un P>t menor que 5% y sus intervalos van de .7 y 2.5, por lo que no puede ser cero, pero sí se paga un poco en la r2 y la r2 ajustada, pero es mejor ya que se evita el ruido en el modelo

#### la constante aunque no parezca significativa, no se debe eliminar